# lister les descriptions

In [ ]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["masque_et_la_plume"]
collection = db["episodes"]

In [ ]:
import pymongo


def list_columns(collection):
    # Utiliser un ensemble pour stocker les noms de colonnes uniques
    columns = set()

    # Parcourir tous les documents de la collection
    for document in collection.find():
        # Ajouter les clés du document à l'ensemble des colonnes
        columns.update(document.keys())

    return list(columns)


list_columns(collection)

['_id', 'nom', 'duree', 'url', 'auteur', 'publié', 'description']

In [ ]:
import pymongo
from itertools import islice


def list_descriptions(collection):
    descriptions = {}

    # Parcourir tous les documents de la collection
    for document in collection.find():
        # Ajouter la description au dictionnaire si elle existe
        if "description" in document:
            descriptions[document["_id"]] = document["description"]
    return descriptions


descriptions = list_descriptions(collection)
dict(islice(descriptions.items(), 5))

{ObjectId('6773e32258fc5717f3516b98'): 'durée : 00:49:48 - Le Masque et la Plume - par : Rebecca Manzoni - Des pieds-nicklés au pays du comté, un film de Paul Schrader avec Richard Gere, le duo reformé d\'"American gigolo", un biopic avec Sandrine Kiberlain sur une grande actrice française, un autre sur Saint-Exupéry, une histoire de femmes... - réalisé par : Guillaume Girault',
 ObjectId('6773e32258fc5717f3516b99'): "durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier - réalisé par : Guillaume Girault",
 ObjectId('6773e32258fc5717f3516b9a'): 'durée : 00:46:52 - Le M

# livres, films, pieces de theatre

In [ ]:
from transformers import pipeline

# Charger le pipeline de classification de texte
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Labels possibles
labels = ["livres", "films", "pièces de théâtre"]


def classify_descriptions(descriptions):
    results = []
    for id, description in descriptions.items():
        result = classifier(description, labels)
        results.append(
            {
                "id": id,
                "description": description,
                "label": result["labels"][0],
                "score": result["scores"][0],
            }
        )
    return results


# Classifier les descriptions
classified_descriptions = classify_descriptions(descriptions)

# Afficher les résultats
for item in classified_descriptions[:5]:
    print(f"Description: {item['id']} {item['description']}")
    print(f"Label: {item['label']} (Score: {item['score']:.2f})")
    print()

Device set to use cpu


Description: 6773e32258fc5717f3516b98 durée : 00:49:48 - Le Masque et la Plume - par : Rebecca Manzoni - Des pieds-nicklés au pays du comté, un film de Paul Schrader avec Richard Gere, le duo reformé d'"American gigolo", un biopic avec Sandrine Kiberlain sur une grande actrice française, un autre sur Saint-Exupéry, une histoire de femmes... - réalisé par : Guillaume Girault
Label: films (Score: 0.83)

Description: 6773e32258fc5717f3516b99 durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier - réalisé par : Guillaume Girault
Label: livres (Score: 0.98)

Description: 67

# conserver en base

In [ ]:
classified_descriptions[:5]

[{'id': ObjectId('6773e32258fc5717f3516b98'),
  'description': 'durée : 00:49:48 - Le Masque et la Plume - par : Rebecca Manzoni - Des pieds-nicklés au pays du comté, un film de Paul Schrader avec Richard Gere, le duo reformé d\'"American gigolo", un biopic avec Sandrine Kiberlain sur une grande actrice française, un autre sur Saint-Exupéry, une histoire de femmes... - réalisé par : Guillaume Girault',
  'label': 'films',
  'score': 0.8343602418899536},
 {'id': ObjectId('6773e32258fc5717f3516b99'),
  'description': "durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier

In [ ]:
# Parcourir la liste des descriptions classifiées
for item in classified_descriptions:
    # Vérifier si l'entrée existe déjà dans la base de données
    existing_entry = collection.find_one({"_id": item["id"]})

    if existing_entry:
        # Vérifier si la colonne 'type' est présente
        if "type" not in existing_entry:
            # Mettre à jour l'entrée existante avec le label et le score
            collection.update_one(
                {"_id": item["id"]},
                {"$set": {"type": item["label"], "type_score": item["score"]}},
            )
            print(f"Type ajouté pour l'ID: {item['id']}")

Type ajouté pour l'ID: 6773e32258fc5717f3516b98
Type ajouté pour l'ID: 6773e32258fc5717f3516b99
Type ajouté pour l'ID: 6773e32258fc5717f3516b9a
Type ajouté pour l'ID: 6773e32258fc5717f3516b9b
Type ajouté pour l'ID: 6773e32258fc5717f3516b9c
Type ajouté pour l'ID: 6773e32258fc5717f3516b9d
Type ajouté pour l'ID: 6773e32258fc5717f3516b9e
Type ajouté pour l'ID: 6773e32258fc5717f3516b9f
Type ajouté pour l'ID: 6773e32258fc5717f3516ba0
